In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import torch
import torch.nn as nn
import torch.optim as optim

import seaborn as sns

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

wDr = os.getcwd()

## Read in dynamic dataset

In [ ]:
concatenated_df = pd.read_csv(wDr + r"\Final_dataset.csv")
concatenated_df

## Data Visualisation

In [ ]:
scaler = MinMaxScaler()

# Scale the velocity column
concatenated_df['velocity'] = scaler.fit_transform(concatenated_df[['velocity']])
concatenated_df

In [ ]:
# Calculate mean
mean = concatenated_df.mean()

# Calculate median
median = concatenated_df.median()

# Calculate mode
mode = concatenated_df.mode().iloc[0]  # mode() can return multiple values, so take the first one

# Calculate min
min_val = concatenated_df.min()

# Calculate max
max_val = concatenated_df.max()

# Combine all statistics into a single DataFrame
stats = pd.DataFrame({
    'Mean': mean,
    'Median': median,
    'Mode': mode,
    'Min': min_val,
    'Max': max_val
})

print(stats)

In [ ]:
num_unique_column1 = concatenated_df['theta_min'].nunique()
num_unique_column2 = concatenated_df['theta_max'].nunique()
print(num_unique_column1, num_unique_column2)

In [ ]:
# Round theta values to 4 decimal places
concatenated_df['theta_min'] = concatenated_df['theta_min'].astype(float).round(4)
concatenated_df['theta_max'] = concatenated_df['theta_max'].astype(float).round(4)
concatenated_df = concatenated_df.apply(lambda col: col.astype(float) if col.dtypes == 'object' else col)
column_types = concatenated_df.dtypes

In [ ]:
# Visualize dataset
plt.scatter(concatenated_df['Peak Stress (MPa)'], concatenated_df['Strain Energy Density (MPa)'])
plt.xlabel('Peak Stress (MPa)')
plt.ylabel('Strain Energy Density (MPa)')
plt.xlim(0, 7)
plt.ylim(0, 0.6)
plt.show()

### Data Histogram

In [ ]:
# Plotting histograms for columns 'A' and 'B'
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.hist(concatenated_df['Peak Stress (MPa)'], bins=10, color='blue', edgecolor='black')
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.title('Histogram of Peak Stress')

plt.subplot(1, 2, 2)
plt.hist(concatenated_df['Strain Energy Density (MPa)'], bins=10, color='red', edgecolor='black')
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.title('Histogram of Strain Energy Density')

plt.tight_layout()
plt.show()

### Defining X and Y

In [ ]:
X = concatenated_df.iloc[:, :5].values
Y = concatenated_df.iloc[:, 5:].values

In [ ]:
# Save dataset
#concatenated_df.to_csv('Data_iteration0.csv')

In [ ]:
# Concatenate X and Y
data = np.concatenate((X, Y), axis=1)

# Create a DataFrame
df = pd.DataFrame(data, columns=[f"X{i}" for i in range(X.shape[1])] + [f"Y{j}" for j in range(Y.shape[1])])

# Calculate correlation matrix
corr_matrix = df.corr()

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap between Inputs and Outputs')
plt.show()

## MLP

In [ ]:
class SimpleMLP(nn.Module):
    def __init__(self, input_size, output_size):
        super(SimpleMLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, output_size)
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
# Splitting data into training and remaining (which includes validation and test)
X_train, X_remaining, Y_train, Y_remaining = train_test_split(X, Y, test_size=0.3, random_state=seed, shuffle=True)

# Splitting remaining data into validation and test sets (50% each of the remaining 30%)
X_val, X_test, Y_val, Y_test = train_test_split(X_remaining, Y_remaining, test_size=0.5, random_state=seed, shuffle=True)

# Check if GPU is available and set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32).to(device)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
Y_val_tensor = torch.tensor(Y_val, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
Y_test_tensor = torch.tensor(Y_test, dtype=torch.float32).to(device)

# Define the neural network
input_size = X_train.shape[1]
output_size = 2

In [ ]:
# Initialize the model
forward_model = SimpleMLP(input_size, output_size).to(device)

In [ ]:
# Define loss function and optimizer
criterion = nn.L1Loss()
#optimizer = optim.Adam(forward_model.parameters(), lr=0.001)
optimizer = optim.Adam(forward_model.parameters(), lr=0.001, weight_decay=1e-5)  # L2 regularization
loss_values, validation_loss, testing_loss = [], [], []

# Training the neural network
num_epochs = 5000 #30000
for epoch in range(num_epochs):
    # Forward pass
    outputs = forward_model(X_train_tensor)
    loss = criterion(outputs, Y_train_tensor)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Store the loss value
    loss_values.append(loss.item())

    # Validation phase (assume X_val_tensor and Y_val_tensor are defined for validation data)
    forward_model.eval()
    with torch.no_grad():
        val_outputs = forward_model(X_val_tensor)
        val_loss = criterion(val_outputs, Y_val_tensor).item()
        validation_loss.append(val_loss)

        test_outputs = forward_model(X_test_tensor)
        test_loss = criterion(test_outputs, Y_test_tensor).item()
        testing_loss.append(test_loss)

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
# Save the model weights after training
#torch.save(forward_model.state_dict(), 'forward_model_weights_5000_iteration0.pth')
print("Model weights saved successfully.")

In [ ]:
# Evaluation
with torch.no_grad():
    Y_pred = forward_model(X_test_tensor)
    mse = mean_squared_error(Y_test_tensor.cpu().numpy(), Y_pred.cpu().numpy())
    print("Mean Squared Error (MSE):", mse)

    # Calculate Mean Absolute Error (MAE)
    mae = mean_absolute_error(Y_test_tensor.cpu().numpy(), Y_pred.cpu().numpy())
    print("Mean Absolute Error (MAE):", mae)

    # Calculate Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mse)
    print("Root Mean Squared Error (RMSE):", rmse)

    # Calculate R-squared (R^2)
    r2 = r2_score(Y_test_tensor.cpu().numpy(), Y_pred.cpu().numpy())
    print("R-squared (R^2):", r2)

    # Calculate Mean Absolute Percentage Error (MAPE)
    mape = np.mean(np.abs((Y_test_tensor.cpu().numpy() - Y_pred.cpu().numpy()) / Y_test_tensor.cpu().numpy())) * 100
    print("Mean Absolute Percentage Error (MAPE):", mape)

# Fix weights for forward model
for param in forward_model.parameters():
    param.requires_grad = False

# Plot the loss function
fig, ax = plt.subplots()

# Adjust tick font sizes
ax.tick_params(axis='both', which='major', labelsize=18)  # Change fontsize for major ticks
ax.tick_params(axis='both', which='minor', labelsize=18)  # Optional: Change fontsize for minor ticks

#plt.xlim(-0.1, 5000)

# Remove the top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.plot(range(num_epochs), loss_values, label='Training Loss', color = 'dodgerblue')
plt.plot(range(num_epochs), testing_loss, label='Testing Loss', color = 'orange')
plt.plot(range(num_epochs), validation_loss, label='Validation Loss', color = 'green')
#plt.plot(np.arange(0, 3530, 1), loss_values, label='Training Loss')
#plt.plot(np.arange(0, 3530, 1), validation_loss, label='Validation Loss')

plt.xlabel('Epochs')
plt.ylabel('Loss')
#plt.title('Loss Function over Epochs')
plt.legend(fontsize=18)
plt.savefig('Supplementary3.png', transparent = True)
plt.show()

#### Performance Evaluation

In [ ]:
# Fix weights for forward model
for param in forward_model.parameters():
    param.requires_grad = False

In [ ]:
Y_pred = forward_model(X_train_tensor)

In [ ]:
# Plotting the best-fit lines
plt.figure(figsize=(12, 6))

# Plotting for each dimension in Y
for i in range(Y_train_tensor.cpu().numpy().shape[1]):
    plt.subplot(1, 2, i+1)
    plt.scatter(Y_train_tensor.cpu().numpy()[:, i], Y_pred.cpu().numpy()[:, i], color='dodgerblue')
    plt.plot([min(Y_train_tensor.cpu().numpy()[:, i]), max(Y_train_tensor.cpu().numpy()[:, i])], [min(Y_train_tensor.cpu().numpy()[:, i]), max(Y_train_tensor.cpu().numpy()[:, i])], color='black', linestyle='--')
    plt.xlabel('Actual Y')
    plt.ylabel('Predicted Y')
    #plt.title(f'Dimension {i+1}')

    # Set tick font sizes
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)

    if i == 0:
        plt.xlim(0, 6)
        plt.ylim(0, 6)
    else:
        plt.xlim(0, 0.6)
        plt.ylim(0, 0.6)
    
    r2 = r2_score(Y_train_tensor.cpu().numpy()[:, i], Y_pred.cpu().numpy()[:, i])
    print(r2)

plt.tight_layout()
#plt.savefig('Supplementary3_1.png', transparent = True)
plt.show()

In [ ]:
Y_pred = forward_model(X_test_tensor)

In [ ]:
# Plotting the best-fit lines
plt.figure(figsize=(12, 6))

# Plotting for each dimension in Y
for i in range(Y_test_tensor.cpu().numpy().shape[1]):
    plt.subplot(1, 2, i+1)
    plt.scatter(Y_test_tensor.cpu().numpy()[:, i], Y_pred.cpu().numpy()[:, i], color='orange')
    plt.plot([min(Y_test_tensor.cpu().numpy()[:, i]), max(Y_test_tensor.cpu().numpy()[:, i])], [min(Y_test_tensor.cpu().numpy()[:, i]), max(Y_test_tensor.cpu().numpy()[:, i])], color='black', linestyle='--')
    plt.xlabel('Actual Y')
    plt.ylabel('Predicted Y')
    #plt.title(f'Dimension {i+1}')

    # Set tick font sizes
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)

    if i == 0:
        plt.xlim(0, 6)
        plt.ylim(0, 6)
    else:
        plt.xlim(0, 0.6)
        plt.ylim(0, 0.6)

    r2 = r2_score(Y_test_tensor.cpu().numpy()[:, i], Y_pred.cpu().numpy()[:, i])
    print(r2)

plt.tight_layout()
#plt.savefig('Supplementary3_2.png', transparent = True)
plt.show()

In [ ]:
from scipy.stats import gaussian_kde

# Loop through each vector and plot its probability density

relative_error_PS = []
relative_error_SED = []

for j in range(Y_pred.shape[0]):

    # Calculate the relative error for the i-th vector
    relative_error_PS.append(np.abs(Y_pred[j][0].cpu() - Y_test_tensor[j][0].cpu()) / Y_test_tensor[j][0].cpu())
    relative_error_SED.append(np.abs(Y_pred[j][1].cpu() - Y_test_tensor[j][1].cpu()) / Y_test_tensor[j][1].cpu())

In [ ]:
# Estimate the probability density

density = gaussian_kde(relative_error_PS)
x = np.linspace(0, np.max(relative_error_PS), 1000)

# Plot the probability density
plt.plot(x, density(x), label=f'Vector {i+1}')

# Calculate %RE less than 10%
samples1, samples2, samples3 = 0, 0, 0
samples1 += len([ele for ele in relative_error_PS if ele < 0.10])
samples2 += len([ele for ele in relative_error_PS if ele < 0.20])
samples3 += len([ele for ele in relative_error_PS if ele < 0.30])


print(samples1/(Y_pred.shape[0]))
print(samples2/(Y_pred.shape[0]))
print(samples3/(Y_pred.shape[0]))


# Customize the plot
plt.title('Probability Density vs. Relative Error for Multiple Vectors')
plt.xlabel('Relative Error')
plt.ylabel('Probability Density')
plt.legend()
plt.grid(True)

# Display the plot
plt.show()

In [ ]:
# Estimate the probability density

density = gaussian_kde(relative_error_SED)
x = np.linspace(0, np.max(relative_error_SED), 1000)

# Plot the probability density
plt.plot(x, density(x), label=f'Vector {i+1}')

# Calculate %RE less than 10%
samples1, samples2, samples3 = 0, 0, 0
samples1 += len([ele for ele in relative_error_SED if ele < 0.10])
samples2 += len([ele for ele in relative_error_SED if ele < 0.20])
samples3 += len([ele for ele in relative_error_SED if ele < 0.30])


print(samples1/(Y_pred.shape[0]))
print(samples2/(Y_pred.shape[0]))
print(samples3/(Y_pred.shape[0]))


# Customize the plot
plt.title('Probability Density vs. Relative Error for Multiple Vectors')
plt.xlabel('Relative Error')
plt.ylabel('Probability Density')
plt.legend()
plt.grid(True)

# Display the plot
plt.show()